# Earthmoving-mobile-crane-CRISP-DM-project

**Author:** _Ming Xu_

**Last update:** _2024.06.24_

In [1]:
# import necessary packages
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from data_processing_program_20240520 import deal_special_brand, pre_processing, data_preperation, matching_program, search_regex, search_capacity, mark_unknown_model_with_exsisted_lifting_capacity, mark_outliers, convert_usd_to_cny, update_regex_df, check_col_names, define_load_interval, define_excavator_load_type_interval, check_col_names, matching_program_individual
pd.options.display.float_format = '{:.1f}'.format
import re

# Data pre-processing

In [2]:
# Read dataframe, and input the route to the following code
df = pd.read_excel('/Users/mingxu/Traymann/ZOOMLION/统计分析报告/8_印度尼西亚/土方/挖掘机/数据处理/842952印尼进口-202401-202403.xls', sheet_name="Sheet1")

Make sure **'date', 'product description', 'supplier', 'amount in usd', 'weight', 'qty', 'unit'** are existed in the col names

In [3]:
# convert the col_names into English
df.columns = ['month', 'hs code', 'product description', 'code description', 'brand', 'importer', 'supplier', 'original country', 'original state',
              'qty', 'unit', 'amount in usd', 'price in usd', 'amount in contract', 'price in contract', 'date', 'declaration number', 'import/export',
              'destination port in indonesia', 'foreign ports of loading', 'importer address', 'exporter address', 'currency', 'amount in IDR', 'price in IDR', 
              'unit price in usd by weight', 'weight in kg', 'weight in tons']

Convert **date** in format

In [4]:
df['date'] = pd.to_datetime(df['date'])

Check units to ensure that they are unified. Exceptions should be checked individually and get unified, e.g., the presence of UNIT and SET in the tables

In [5]:
df['unit'].value_counts()

Number of international units    1107
Set                                97
Piece                              14
pack                                1
Name: unit, dtype: int64

Delete rows that **'amount in usd'** and **'price in usd'**amount in usd **below 10K**.

In [6]:
df = df[df['amount in usd']>=10000]
df = df[df['price in usd']>=10000]
df = df.reset_index(drop=True)

In [7]:
df['unit'] = 'UNT'

Calculate unit price and net weight in tons

$$unit\ price = \frac{amount}{quantity}$$

$$weight\ (ton) = \frac{weight\ (kg)}{quantity \times 1000}$$

In [8]:
# Converts quantities and weights into numeric format for easy calculation
df['qty'] = pd.to_numeric(df['qty'], errors='coerce')
df['weight in kg'] = pd.to_numeric(df['weight in kg'], errors='coerce')

# Calculate unit price and net weight per unit in ton
df['price in usd'] = df['amount in usd']/df['qty']
df['unit weight in ton'] = df['weight in kg']/df['qty']/1000

In [ ]:
# load reference dataframe
df_ref = pd.read_excel('/Users/mingxu/Traymann/ZOOMLION/统计分析报告/8_印度尼西亚/土方/挖掘机/数据处理/数据对照表/型号对照/20240115_土方对照表_含吨位和马力.xlsx', sheet_name="对照表")

**data_preperation(df, df_ref) definition**

* input parameter 输入参数
    * Original dataframe 原始数据表
    * reference dataframe 数据对照表
* Output 输出
    * Creat a copy of **product description** to avoid the direct modification on original data 新建**产品描述**副本以免影响原始数据；
    * Remove the special characters, space and puntuations in product description 去除描述中存在的特殊字符及空格

In [ ]:
data_preperation(df, df_ref)

# Matching the reference table

Goal: mathcing the exsisted knowledge in the reference table to the processing data.

目的：将对照表里存在的设备信息匹配到目前正在处理的数据中

**matching_program() definition:**
* Input parameters (in order): original dataframe, reference table, file type ('crane' or 'excavator') 

    输入参数（依次）：原始数据表，对照表，文件类型。其中文件类型可填写 'crane' 或 'excavator'
* Output: processed dataframe; error indexes 

    输出：匹配后的表格；报错的行号（若有）

**What happened? and what's the standard? 运行机制及标准：**

**1. Mapping the existed knowledge in reference table to the original dataframe 将数据对照表里存在的信息对应关系匹配到原始数据里**
* Check if any brand existed in product description column -> Mark: Brand in description; Only take the first brand if multipul brands existed
* Check if any brand existed in supplier description column -> Mark: Brand in supplier; Only take the first brand if multiple brands existed
* For rows with brands -> Searching the brand in reference table -> Check if the model existed in product description -> if only a single model match -> Mark: Fully match
* For rows with brands -> Searching the brand in reference table -> Check if the model existed in product description -> if several models match, only take the longest model -> Mark: Multiple model matched, take the longest match
* Nothing matched -> Mark: No match

    <details>
        <summary>CN VER.</summary>
    
        * 对照表里的品牌是否存在于产品描述里 -> 存在标记品牌；备注：描述中存在品牌；存在多个匹配品牌时，取第一个品牌
    
        * 对照表里的品牌是否存在于供应商描述里 -> 存在标记品牌；备注：供应商存在品牌；存在多个匹配品牌时，取第一个品牌
    
        * 存在品牌的条目 -> 对照表筛选该品牌 -> 标记匹配的型号；型号唯一时；备注：完美匹配
    
        * 存在品牌的条目 -> 对照表筛选该品牌 -> 标记匹配的型号；有多个型号匹配时，取最长的型号；备注：多个型号匹配，取最长的型号
    
        * 没有匹配的品牌 -> 将在下一个步骤通过通用表达式对条目进行搜索
    
        * 什么都没有的时候 -> 备注：无品牌型号匹配


**2. Delete the irrelevant items 删除包含不相关关键词的条目**

    Current statistics for irrelevant keywords include the following 目前统计不相关关键词含如下：

    ['CARRIER', 'TELESCOPLADER', 'HARBOUR', 'OPEN SHEET', 'STACK', 'BOAT', 'BACKHOE', 'SKID', 'ROLLER', 'BENZ', 'TELEHANDLER', 'LOADER', 'FORK', 'PAVER', 'STACKER', 'MATERIAL HANDLER', 'BRIDGE', 'REACH', 'HANDER', 'GRABBER', GANTRY', 'BACK HOE', 'PORT', 'MERCEDES', 'VİNCE', 'SPIDER', 'PIPE', 'HANDLING', 'GLASS CRANE', 'LOAD', 'GRADER', 'GLASS CRANE', 'SPIDER']
                            
**3. Infer product type according to keywords in description 根据货物描述包含的关键词判断产品类型**
* Excavator 挖掘机：
    * crawler excavator -> EXCAVATOR
    * wheel 或 tire -> WHEEL EXCAVATOR
    * AMPHIBIOUS -> AMPHIBIOUS EXCAVATOR
    * without the above keywords -> UNKNOWN
* Mobile Crane 工起：
    * Wheel **or** tire -> WHEELED CRANE
    * rough -> ROUGH-TERRAIN CRANE
    * crawler -> CRAWLER CRANE
    * crawler **and** telescopic -> CRAWLER CRANE (TELESCOPIC_BOOM)
    * without the above keywords -> UNKNOWN
    
**4. New or used unit judgement 判断新旧机**
* Contain the following keywords: ['USED', 'SECOND HAND', 'SECONDHAND', 'OLD', '2ND HAND', 'REFURBISH']
* Contain a number ranging from 1950 to 2019

    **Mark as used unit**

**5. Parts**
* Contain the following keywords: ['CKD', 'SKD', 'partial'] Mark as **‘Parts’**

In [11]:
# input parameters: original dataframe, reference table, file type (fill in 'crane' or 'excavator')
# output: df with matched info; error row index
df, error_rows_index = matching_program(df, df_ref, file_type='excavator')

In [12]:
# check if contain error rows
error_rows_index.keys()

dict_keys([])

In [13]:
# check the remarks
df['remark'].value_counts()

Fully match                          593
Brands existed but without models     73
No match                              65
Parts                                 16
Name: remark, dtype: int64

# Matching with regular expression
Goal: For data that is no labeled, labeling them by regular expression based on naming rules. 

目的：根据各品牌的型号命名规则，对上一步没有匹配上的数据进行规则匹配并进行信息标记

In [14]:
# Load regex reference table
df_regex = pd.read_excel('/Users/mingxu/Traymann/ZOOMLION/统计分析报告/8_印度尼西亚/土方/挖掘机/数据处理/数据对照表/Regex/20240527_土方REGEX对照表.xlsx',sheet_name='对照表')

**search_regex() definition**
* Original dataframe 原始表格
* Filter condition(s) 筛选条件
* Searching col 搜索列的列名
* regex reference table 通用表达式对照表
* Filter brands or not 是否对品牌进行筛选 **filter_brand=True/False**
    * True: If the items has been flagged with brand, this function will **only search regexes within the flagged brand**
    * False: Ignore the flagged brand and search through the whole reference table
* Whether to mark working capacity (e.g. tonnage/max lift) **mark_load=True/False**
    * True: mark
    * False: ignore

In [16]:
# Aim to have a second search for "Brands existed but without models" and "No match" items
df = search_regex(df, df['remark'].isin(['Brands existed but without models']), 'description2', df_regex, filter_brand=True, mark_load=True)

In [17]:
df = search_regex(df, df['remark'].isin(['No match']), 'description2', df_regex, filter_brand=False, mark_load=True)

**search_capacity() definition**
* Original dataframe 原始表格
* Filter condition(s) 搜索条件
* Searching col 搜索列名
* regex 搜索规律通用表达式

目的：检索描述中包含 **$数字 + (metric) \ TONS$** 关键词。这部分将会把TONS前面的数字标记为吊重。

Goal: Searching the pattern of **$number + (metric) \ TONS$** in description, and mark the number as maximum lifting capacity

This part of data will be marked as: _Description contains working capacity_

In [18]:
condition = df['capacity']=='UNKNOWN'
search_col = 'description2'
capacity_regex = r'\b(\d+(?:\.\d+)?)\s*(?:METRIC\s*)?TONS?\b'
# (?:METRIC\s*)?: Matches an optional 'METRIC' followed by optional whitespace characters. 
# The (?: ... ) is a non-capturing group.
df = search_capacity(df, condition, search_col, capacity_regex)

In [19]:
df.loc[df['remark'].isin(['Brands existed but without models']),'brand'].unique()

array(['MAXPOWER', 'LONKING'], dtype=object)

In [20]:
df['remark'].value_counts()

Fully match                                                   593
Unique model match with regex                                  45
No match                                                       39
No brand in description, and unique model match with regex     25
Keep the longest from the multiple matched                     23
Parts                                                          16
Brands existed but without models                               5
Description contains working capacity                           1
Name: remark, dtype: int64

Crawler crane 300T以上的基本可以判断为 Lattice boom

In [21]:
df['capacity'] = pd.to_numeric(df['capacity'], errors='coerce')
# df.loc[(df['类型'].isin(['CRAWLER CRANE']))&(df['吨位'].notna())&(df['吨位']>300),'类型'] = 'CRAWLER CRANE (LATTICE_BOOM)'

Since CATERILLAR's naming rules is relatively simple and can easily be mislabeled by serial numbers, this issue should be  addressed separately.

In [22]:
df.loc[df['brand']=='CATERPILLAR',['model','type','new/used','remark']].value_counts()

model    type             new/used  remark     
305.5E2  EXCAVATOR        new       Fully match    12
320      EXCAVATOR        new       Fully match     7
330 GC   EXCAVATOR        new       Fully match     7
313      EXCAVATOR        new       Fully match     6
307      EXCAVATOR        new       Fully match     3
6020B    EXCAVATOR        new       Fully match     3
345 GC   EXCAVATOR        new       Fully match     2
333      EXCAVATOR        new       Fully match     1
M320     WHEEL EXCAVATOR  new       Fully match     1
dtype: int64

In [23]:
cat_regex = pd.read_excel('/Users/mingxu/Traymann/ZOOMLION/数据对照表/20240312_土方CAT_REGEX对照表.xlsx',sheet_name='Sheet1')

In [24]:
df = search_regex(df, (df['brand']=='CATERPILLAR')&(df['model']=='UNKNOWN'), 'description2', cat_regex, filter_brand=True, mark_load=True)

In [25]:
df['remark'].value_counts()

Fully match                                                   593
Unique model match with regex                                  45
No match                                                       39
No brand in description, and unique model match with regex     25
Keep the longest from the multiple matched                     23
Parts                                                          16
Brands existed but without models                               5
Description contains working capacity                           1
Name: remark, dtype: int64

# Infer and label the items with existed information

In this section, I will use the labeled items which with the crucial information of ['brand','model','capacity','type'], such as _item A_ to inference the missing info of _item B_. 

Condition: If item A's brand and type are consistent to item B, and its capacity is within the range of ±5% of that of item B.

For instance: A contains all information. B's working capacity is infered from the description via search the patter of number + tons. This function will take ±5% as a threshold, and if A's capacity just fall in this range, then I will label B's model by A.

| item | brand | model | capacity | type |
|------|------|------|------|------|
| A    | ✓    | ✓    | ✓    | ✓    |
| B    | ✓    | ✘    | ✓    | ✓    |

<details>
    <summary>CN VER.</summary>
此部分将用已标记了品牌、型号、吊重、类型信息的条目（A条目），反向匹配包含品牌、吊重和类型的条目（B条目）。

运行条件：只要满足品牌和类型匹配，且B的吊重在已知A吊重的±5%范围内。

例如：已知A和B两个条目品牌和类型一致，A包含上述的4个已知信息。B的吊重通过数字+TONS的方式知道该信息。下面的方程将会计算B的吨位±5%作为阈值，如果A的吊重正好在这个区间内，那判断B的型号和A一致。

In [26]:
df = mark_unknown_model_with_exsisted_lifting_capacity(df)

In [27]:
# Calculate the amount and quantity of no matched items
unlabeled_value = round(df.loc[df['remark'] == 'No match','amount in usd'].sum(),1)
unlabeled_qty = df.loc[df['remark'] == 'No match','qty'].sum()
print(f'No match items amount: $ {round(unlabeled_value,2)}')
print(f'No match items qty: {unlabeled_qty} units')

No match items amount: $ 8756612.0
No match items qty: 152 units


# Mark the outliers

Check if N/A value contains

In [28]:
df['unit weight in ton'].isna().sum()

260

**mark_outliers(df, term=True) definition**
* Input paramter:
    * original dataframe
    * term: choose from **True/False**; Note: If the data contains shipping terms, true could be filled, and make sure there is a column "term" in dataframe
        * True: The outlier judgment will be based on the brand, model, and terms as the filtering conditions, and the **median price** in the case that the above three marks are satisfied at the same time will be taken as the reference standard;
        * False：If the original dataframe does not contain terms, fill in false will ignore terms and **will only take the median price under the same brand and model** according to the above criteria.

**Goal:** Find out the outlier items for further price analysis

**Price outlier judgement criteria:**

Calculate the median price of models under the same brand, if trade terms exist, make a judgment under the same trade terms
* Unknown: unknown model, used machine;
* Normal: item prices within median ± median*20%;
* Outlier: prices outside the aforementioned threshold.

In [29]:
df = mark_outliers(df, term=False)

# Working capacity interval division
**define_load_interval(df, file_type, load_interval)**

Input parameters:

* Original dataframe
* File_type: should fill in **crane** or **excavator**; Notice: the interval definition differs from file types
* Define the load interval
   * For example: define the interval as the specified number. If fill in 50, I will return ≤50T, 50-100T, 100-150T, and so forth.

**Mobile crane**

* There are cases where the lifting capacity is much larger than the self-weight of the equipment itself, and it varies from types, so it can't speculate by the weight of the equipment itself;
* Will only mark the interval of items that are labeled with lifting capacity.
    
    
**Excavator**

The work load of excavator is almost identical to the weight of itself, and for unknown items will be divided by weight
* Define the interval of known items
* Define the interval of unknown items by weight

In [30]:
define_load_interval(df, file_type = 'excavator', load_interval = 10)

In [31]:
define_excavator_load_type_interval(df)

In [33]:
df['type interval'].value_counts()

5-10T      300
10-30T     265
30-70T     119
<5T         27
≥90T        15
UNKNOWN     15
70-90T       6
Name: type interval, dtype: int64

In [34]:
df['capacity interval'].value_counts()

5-10T       300
20-30T      222
30-40T       61
10-20T       43
40-50T       41
<5T          27
50-60T       16
UNKNOWN       7
110-120T      6
70-80T        5
200-210T      4
230-240T      3
80-90T        1
90-100T       1
100-110T      1
60-70T        1
Name: capacity interval, dtype: int64

In [35]:
df['type interval'] = df['type interval'].fillna('UNKNOWN')
df['capacity interval'] = df['capacity interval'].fillna('UNKNOWN')

# Exchange rate conversion

**convert_usd_to_cny(df, rate_dict)**

Input parameters:
* Original dataframe: make sure ['date', 'price in usd', 'amount in usd'] columns are in dataframe
* Exchange rate dictionary: a dictionary file, and should fill in the latest USD-CNY rate to "rate_dict". Historical data will be calculated automatically based on the date.

In [36]:
rate_dict = {2023: {1: 6.7604, 2: 6.9519, 3: 6.8717, 4: 6.924, 5: 7.0821, 6: 7.2258, 7: 7.1305, 8: 7.1811, 9: 7.1798, 10: 7.1779, 11: 7.1018, 12: 7.0827},
             2024: {1: 7.1039, 2: 7.1036, 3: 7.0950, 4:7.1063}}

df = convert_usd_to_cny(df, rate_dict)

In [37]:
# remove the extra marks
df['model'] = df['model'].apply(lambda x: x.replace(' ',''))
df['model'] = df['model'].apply(lambda x: x.lstrip())
df['model'] = df['model'].apply(lambda x: x.rstrip())

**The code below only use for mobile crane data processing**
* Merge the tranlation to dataframe

In [ ]:
EN_CN = pd.read_excel('/Users/mingxu/Traymann/ZOOMLION/数据对照表/工起类型分类对照表.xlsx',sheet_name='Sheet1')

In [ ]:
df = pd.merge(df, EN_CN, how='left', on='类型')

Save the file

In [ ]:
df.to_excel('/Users/mingxu/Traymann/ZOOMLION/统计分析报告/8_印度尼西亚/土方/挖掘机/数据处理/2024年一季度印尼挖掘机进口数据_已清理和异常判断.xlsx', sheet_name='Sheet1', index=False)

# Update historical data

Concatenate the historical to the latest.
* Make sure the processing logic are aligned;
* Make sure the column names of both dataframes are identical.

In [ ]:
df_hist = pd.read_excel('/Users/mingxu/Traymann/ZOOMLION/统计分析报告/8_印度尼西亚/土方/挖掘机/2022年8-2023年印尼挖掘机进口数据快报.xlsx',sheet_name='7_原始数据')

**check_col_names(df.columns, df_hist.columns)**

Check the identity

Check that the column names of the merged tables are consistent, with ORIGINAL on the left and REFERENCE on the right; as long as it is satisfied that the column names of one of the files are a subset of another file

In [ ]:
check_col_names(df.columns, df_hist.columns)

Concatenate historical data

In [ ]:
df_updated = pd.concat([df_hist, df], axis=0, ignore_index=True)

In [ ]:
df_updated['model'] = df_updated['model'].apply(lambda x: x.replace(' ',''))
df_updated['model'] = df_updated['model'].apply(lambda x: x.lstrip())
df_updated['model'] = df_updated['model'].apply(lambda x: x.rstrip())

In [ ]:
# Save file
df_updated.to_excel('/Users/mingxu/Traymann/ZOOMLION/统计分析报告/8_印度尼西亚/土方/挖掘机/2022年8-2024年3月印尼挖掘机进口数据_已清理及异常判断.xlsx',sheet_name='Sheet1',index=False)